In [2]:
from paddleocr import PaddleOCR, draw_ocr
from PIL import Image
import os

# 初始化 OCR（英文模型）
ocr = PaddleOCR(use_angle_cls=False, lang='en')  # 英文模型

# 圖片路徑
img_path = r'C:\Users\zhuang\Desktop\Paddle\engimg\id_card_images\image_00000.png'

# 執行 OCR
result = ocr.ocr(img_path, det=True, rec=True)

# 印出文字結果
print("OCR 辨識結果：")
for line in result[0]:
    print(f"位置：{line[0]}")
    print(f"文字：{line[1][0]}，信心分數：{line[1][1]}")
    print('-' * 30)

# 畫出辨識框與文字
image = Image.open(img_path).convert('RGB')
boxes = [line[0] for line in result[0]]
txts = [line[1][0] for line in result[0]]
scores = [line[1][1] for line in result[0]]

# ✅ 指定英文字型，避免 OSError
font_path = r'C:\Windows\Fonts\arial.ttf'

im_show = draw_ocr(image, boxes, txts, scores, font_path=font_path)
im_show = Image.fromarray(im_show)

# 儲存結果圖片
output_img_path = os.path.join(os.path.dirname(img_path), 'image_00000_ocr_result.jpg')
im_show.save(output_img_path)

print(f"\n✅ 辨識結果圖片已保存到：{output_img_path}")


[2025/05/25 17:54:50] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\zhuang/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\zhuang/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_score_th

In [6]:
import editdistance

def cer(gt, pred):
    return editdistance.eval(gt, pred) / len(gt)

# 兩組字串
gt = "The experimental protocol was approved by the Animal Ethics Committee"
noisy = "Theexperimentalprotocol was appr  oved by   th  eAnimal EthicsCommittee"

print(f"Ground Truth: {gt}")
print(f"Noisy Text  : {noisy}")

print(f"CER (含空格): {cer(gt, noisy):.3f}")
print(f"CER (去空格): {cer(gt.replace(' ', ''), noisy.replace(' ', '')):.3f}")


Ground Truth: The experimental protocol was approved by the Animal Ethics Committee
Noisy Text  : Theexperimentalprotocol was appr  oved by   th  eAnimal EthicsCommittee
CER (含空格): 0.130
CER (去空格): 0.000


In [7]:
from paddleocr import PaddleOCR
import editdistance

def cer(s1, s2):
    return editdistance.eval(s1, s2) / max(len(s1), len(s2))

ground_truth = "The experimental protocol was approved by the Animal Ethics Committee"
noisy_text = "The experimental protocol ( was approved by the Animal Ethics Committee"

# 計算 CER
error_rate = cer(ground_truth, noisy_text)

print("Ground Truth:", ground_truth)
print("Noisy Text  :", noisy_text)
print(f"CER: {error_rate:.3f}")

# 顯示差異位置（簡單比對）
import difflib
diff = difflib.ndiff(ground_truth, noisy_text)
print("\n差異位置:")
print(''.join(diff))


Ground Truth: The experimental protocol was approved by the Animal Ethics Committee
Noisy Text  : The experimental protocol ( was approved by the Animal Ethics Committee
CER: 0.028

差異位置:
  T  h  e     e  x  p  e  r  i  m  e  n  t  a  l     p  r  o  t  o  c  o  l+  + (     w  a  s     a  p  p  r  o  v  e  d     b  y     t  h  e     A  n  i  m  a  l     E  t  h  i  c  s     C  o  m  m  i  t  t  e  e


In [13]:
from paddleocr import PaddleOCR
from PIL import Image
import os

# 初始化 OCR（英文模型）
ocr = PaddleOCR(use_angle_cls=False, lang='en')

# 圖片資料夾
img_dir = r'C:\Users\zhuang\Desktop\Paddle\engimg\id_card_images'

# OCR 文字結果存放資料夾
txt_dir = r'C:\Users\zhuang\Desktop\Paddle\engimg\ocrtxt'
os.makedirs(txt_dir, exist_ok=True)  # 如果資料夾不存在就建立

# 取得資料夾中所有圖片檔（可自行增加其他副檔名）
valid_exts = ['.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff']
all_images = [f for f in os.listdir(img_dir) if os.path.splitext(f)[1].lower() in valid_exts]

# 只處理前100張
images_to_process = all_images[:100]

for img_name in images_to_process:
    img_path = os.path.join(img_dir, img_name)
    print(f"Processing: {img_name}")
    
    # OCR
    result = ocr.ocr(img_path, det=True, rec=True)
    
    # 將結果文字整合（只保留純文字）
    lines = [line[1][0] for line in result[0]]
    
    # 輸出文字檔案，副檔名改為 .txt
    txt_path = os.path.join(txt_dir, os.path.splitext(img_name)[0] + '.txt')
    with open(txt_path, 'w', encoding='utf-8') as f:
        f.write('\n'.join(lines))

print("✅ OCR 文字結果處理完成（已移除信心分數）！")


[2025/05/25 20:39:44] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\zhuang/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\zhuang/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_score_th

In [14]:
import os
import editdistance

# 資料夾路徑
gt_dir = r'C:\Users\zhuang\Desktop\Paddle\engimg\ground_truth'
ocr_dir = r'C:\Users\zhuang\Desktop\Paddle\engimg\ocrtxt'

# 取得 ground truth 的檔案清單（如：answer_00000.txt）
gt_files = sorted([f for f in os.listdir(gt_dir) if f.startswith('answer_') and f.endswith('.txt')])

total_cer = 0
total_chars = 0

print("每個檔案 CER 計算結果（忽略空白與換行）:")
print("=" * 50)

for gt_file in gt_files:
    idx = gt_file.replace("answer_", "").replace(".txt", "")
    ocr_file = f"image_{idx}.txt"

    gt_path = os.path.join(gt_dir, gt_file)
    ocr_path = os.path.join(ocr_dir, ocr_file)

    if not os.path.exists(ocr_path):
        print(f"❌ 找不到對應 OCR 檔案: {ocr_file}")
        continue

    # 讀取文字內容，忽略空白與換行
    with open(gt_path, 'r', encoding='utf-8') as f:
        gt_text = f.read().replace(" ", "").replace("\n", "")
    with open(ocr_path, 'r', encoding='utf-8') as f:
        ocr_text = f.read().replace(" ", "").replace("\n", "")

    # 計算 CER（edit distance / 真實文字長度）
    dist = editdistance.eval(gt_text, ocr_text)
    char_count = max(len(gt_text), 1)  # 避免除以0
    cer = dist / char_count

    print(f"{gt_file} vs {ocr_file} -> CER: {cer:.3f}")

    total_cer += dist
    total_chars += char_count

# 統計整體 CER
overall_cer = total_cer / total_chars if total_chars > 0 else 0
print("\n" + "=" * 50)
print(f"✅ 總體 CER（忽略空白與換行）: {overall_cer:.3f}")


每個檔案 CER 計算結果（忽略空白與換行）:
answer_00000.txt vs image_00000.txt -> CER: 0.050
answer_00001.txt vs image_00001.txt -> CER: 0.024
answer_00002.txt vs image_00002.txt -> CER: 0.011
answer_00003.txt vs image_00003.txt -> CER: 0.019
answer_00004.txt vs image_00004.txt -> CER: 0.018
answer_00005.txt vs image_00005.txt -> CER: 0.015
answer_00006.txt vs image_00006.txt -> CER: 0.007
answer_00007.txt vs image_00007.txt -> CER: 0.037
answer_00008.txt vs image_00008.txt -> CER: 0.036
answer_00009.txt vs image_00009.txt -> CER: 0.007
answer_00010.txt vs image_00010.txt -> CER: 0.019
answer_00011.txt vs image_00011.txt -> CER: 0.034
answer_00012.txt vs image_00012.txt -> CER: 0.035
answer_00013.txt vs image_00013.txt -> CER: 0.024
answer_00014.txt vs image_00014.txt -> CER: 0.028
answer_00015.txt vs image_00015.txt -> CER: 0.051
answer_00016.txt vs image_00016.txt -> CER: 0.051
answer_00017.txt vs image_00017.txt -> CER: 0.050
answer_00018.txt vs image_00018.txt -> CER: 0.017
answer_00019.txt vs image_

In [8]:
import os
import editdistance
import difflib

# 資料夾路徑
gt_path = r'C:\Users\zhuang\Desktop\Paddle\engimg\ground_truth\answer_00009.txt'
ocr_path = r'C:\Users\zhuang\Desktop\Paddle\engimg\ocrtxt\image_00009.txt'

# 讀取並清理字串（移除空白與換行）
with open(gt_path, 'r', encoding='utf-8') as f:
    gt_text = f.read().replace(" ", "").replace("\n", "")
with open(ocr_path, 'r', encoding='utf-8') as f:
    ocr_text = f.read().replace(" ", "").replace("\n", "")

# 計算 CER
dist = editdistance.eval(gt_text, ocr_text)
cer = dist / max(len(gt_text), 1)

print(f"\n✅ CER: {cer:.3f}\n")

# 顯示差異
print("🔍 Ground Truth:")
print(gt_text)
print("\n🔍 OCR Result:")
print(ocr_text)

# 顯示字元差異（用 difflib）
print("\n🧪 差異位置 (用 difflib 指出錯誤字):")
matcher = difflib.ndiff(gt_text, ocr_text)
for i, s in enumerate(matcher):
    if s.startswith('- ') or s.startswith('+ '):
        print(f"{i:3}: {s}")



✅ CER: 0.007

🔍 Ground Truth:
Q:Theseresultssuggestthatsuchcommunity-basedoutreachprogramscansignificantlyimprovepatientoutcomeandprovidesubstantialcostsavingsforlocalgovernmentsandhospitals.Correlationofparasitealleleswithinavillagerelativetoallelesdrawnatrandomfromtheoverallwormpopulation(parasitegeneticdifferentiationbetweenvillages)privatevoidDemoVideo_MediaEnded(objectsender,RoutedEventArgse)stickwithmebitches.sub.2productsformedinalaserdischarge.Mastika(AugustEdition)hadsomestoriesofhowtheNegroaffectingourlifehereinMalaysia.Itisinterestingtonotethewiderangeofmicrofilarialgenotypedistributionsthatcanbegeneratedbythenullmodel.Asatrialrun,theduotrekkedupBukitTimahhilltwodaysbeforetheirMountKinabaluclimb.Interestingideaaboutthebombsendingthemaincharactersforwardintime.Supportingthatideaisthefactthattheyusedtheexactsamescreeneffectwhenwearemeanttobelievethebombgoesoffaswhenwehavetraveledintimepreviously.ShouldIworkdirectlyattheshaderlevelorinjavascript?Doyonsaidawarranthadbeenissuedf